Setting up a Convolutional Neural Network for image classification with Keras:

Used dataset: Traffic signs: kaggle.com/valentynsichkar/traffic-signs-preprocessed



In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from PIL import Image
import keras

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\gwehrm\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\gwehrm\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\gwehrm\AppData\Roaming\P

## Data

In [2]:
#load the data
data = pickle.load(open("data/data0.pickle","rb"))

In [3]:
# the data is a dictionary with the following keys:
# The dataset is already nicely prepared, therefore, no more preparation is needed. Even the splits
# are already done!
data.keys()

dict_keys(['x_test', 'y_validation', 'x_validation', 'labels', 'x_train', 'y_test', 'y_train'])

In [4]:
# extract the datasets
x_test = data["x_test"]
y_validation = data["y_validation"]
x_validation = data["x_validation"]
labels = data["labels"]
x_train = data["x_train"]
y_test = data["y_test"]
y_train = data["y_train"]

In [5]:
# delete the now obsolete original dataset
del(data)

In [6]:
# the labels are not yet in a one-hot encoding 
display(y_test[1])

# the labels are integers - therefore, the keras function to_categorical can be used to encode it
from keras.utils import to_categorical

# transformation:
def ohe(labs):
    ohe_labs = to_categorical(labs)
    return ohe_labs
y_test = ohe(y_test)
display(y_test[1])

11

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [7]:
# Nice, and the same with the others:
y_validation = ohe(y_validation)
y_train = ohe(y_train)

In [8]:
# the shape of the training data
x_train.shape

(86989, 3, 32, 32)

In [9]:
x_train = np.transpose(x_train/255.,[0,2,3,1])
x_test = np.transpose(x_test/255.,[0,2,3,1])
x_validation = np.transpose(x_validation/255.,[0,2,3,1])

In [10]:
# total number of classes:
len(y_train[1])

43

## Model building

In [28]:
# Import the necessary components from Keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout

# Initialize the model object
model = Sequential()

# Add a convolutional layer
model.add(Conv2D(10, kernel_size=3, activation='relu', 
               input_shape=(32,32,3),
                padding = "same"))

# model.add(Dropout(0.2))


model.add(Conv2D(10, kernel_size=6, activation='relu'))

# model.add(Dropout(0.2))

model.add(Conv2D(10, kernel_size=9, activation='relu'))

# model.add(Dropout(0.2))

# model.add(Conv2D(10, kernel_size=12, activation='relu'))


# Flatten the output of the convolutional layer
model.add(Flatten())
# Add an output layer for the categories
model.add(Dense(43, activation='softmax'))

In [29]:
# Compile the model 
model.compile(optimizer="adam", 
              loss="categorical_crossentropy", 
              metrics=["accuracy"])

# Fit the model on a training set
model.fit(x_train, y_train, 
          validation_split=0.2, 
          epochs=3, batch_size=100)

Train on 69591 samples, validate on 17398 samples
Epoch 1/3
69591/69591 [==============================] - 160s 2ms/step - loss: 0.6421 - accuracy: 0.8505 - val_loss: 0.1501 - val_accuracy: 0.9678
Epoch 2/3
69591/69591 [==============================] - 163s 2ms/step - loss: 0.1008 - accuracy: 0.9753 - val_loss: 0.0911 - val_accuracy: 0.9775
Epoch 3/3
69591/69591 [==============================] - 166s 2ms/step - loss: 0.0538 - accuracy: 0.9864 - val_loss: 0.0728 - val_accuracy: 0.9837


In [30]:
# evaluate the model:
model.evaluate(x_validation,y_validation)

4410/4410 [==============================] - 2s 504us/step


[0.3718595132316186, 0.940362811088562]

In [20]:
# Get the first convolutional layer from the model
c1 = model.layers[0]

# Get the weights of the first convolutional layer
weights1 = c1.get_weights()

# Pull out the first channel of the first kernel in the first layer
kernel = weights1[0][...,0, 0]
print(kernel)

[[ 0.15697241 -0.0315214   0.24909198]
 [-0.00220037 -0.01173912 -0.04497962]
 [ 0.16014211  0.14631014  0.10557701]]


In [22]:
import matplotlib.pyplot as plt

# Convolve with the fourth image in test_data
out = keras.convolution(x_test[3, :, :, 0], kernel)

# Visualize the result
plt.imshow(out)
plt.show()

AttributeError: module 'keras' has no attribute 'convolution'